In [ ]:
import pandas as pd
import numpy as np
import datanoutlibD1 # TODO: kan vi finde på et bedre navn til modulet?

# TODO: Kan alle nedenstående loades og udstilles af datanoutlib, 
# for at minimere antal biblioteker elever skal overskue?

%matplotlib inline  
import matplotlib.pyplot as plot
from pandas.plotting import scatter_matrix # ikke nødvendig længere
from sklearn.base import BaseEstimator, TransformerMixin # bruges vist ikke
from sklearn.model_selection import train_test_split # ikke nødvendig længere
from sklearn.model_selection import StratifiedShuffleSplit # flyttes til vores bibliotek
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from scipy import stats

Del 1: På opdagelse i huspris-datasættet
---------------------------------------

(Jeg tænker måske vi skal lave 2-3 forskellige Notebooks, med hver sin del - den første mest datavisualisering fx)

In [ ]:
# Dette skal eleverne skrive
housing = datanoutlibD1.download_housing_data()
housing.head()

In [ ]:
# Dette skal eleverne skrive
housing["median_income"]

In [ ]:
# Dette skal eleverne skrive
housing["median_income"].plot.hist()

In [ ]:
# Dette skal eleverne skrive - ændr antal bins
housing["median_income"].plot.hist(bins=50)

In [ ]:
# Dette skal eleverne skrive - kig på andre features
housing["housing_median_age"].plot.hist(bins=50)

In [ ]:
# Dette skal eleverne skrive - kig på andre features
housing["households"].plot.hist(bins=50)

In [ ]:
# Dette skal eleverne skrive - kig på andre features
housing["latitude"].plot.hist(bins=50)
plot.show() # hvis elever laver flere plots i samme celle skal de adskilles på denne måde, måske
            # skal vi bare få dem til altid at skrive plot.show() ??
housing["longitude"].plot.hist(bins=50)
plot.show()

In [ ]:
# Vi skaber nye attributes udfra kombinitioner af interessante attributer 
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

Eksempel på opgaver: 
 - Hvad kan vi se ud fra histogram over median indkomst?
 - Hvad kan vi se ud fra histogram over huses alder?
 - Kan vi sige noget om sammenhængen mellem huspris og features ind til videre? (Nej, vi har bare set fordelinger)
 - Kan man se noget ud fra GPS-koordinater (latitude/longitude) på disse histogramer? (Leder videre til næste udforskning hvor plottes på kort)

Note: Nedenstående tænker jeg er faserne eleven går igennem ift. plotting af kortet, de retter bare i en enkelt celle, mens det her er præsenteret som en række celler, så man kan se forskellene i løbet af deres vej igennem

In [ ]:
# de 3 plots er gode eksempler på hvordan man kan lære mere
# ved at visualisere ens data på forskellige måder
housing.plot.scatter(x="longitude", y="latitude")

In [ ]:
# ændr størrelsen på figuren
housing.plot.scatter(x="longitude", y="latitude", figsize=(14,10))

In [ ]:
housing.plot.scatter(x="longitude", y="latitude", figsize=(14,10), alpha=0.1)

In [ ]:
# Ændr størrelsen på punkter
housing.plot.scatter(x="longitude", y="latitude", figsize=(14,10), alpha=0.1, s=50)

In [ ]:
# Ændr størrelsen af punkter til at svare til befolkningen
housing.plot.scatter(x="longitude", y="latitude", figsize=(14,10), alpha=0.1, s=housing["population"])

In [ ]:
# Ændr størrelsen til at svare til befolkningen - skaler ned i størrelse
housing.plot.scatter(x="longitude", y="latitude", figsize=(14,10), alpha=0.1, s=housing["population"]/100)

In [ ]:
# Vis huspris i plottet
housing.plot.scatter(x="longitude", y="latitude", figsize=(14,10), alpha=0.4,
    s=housing["population"]/100,
    c="median_house_value", colormap="jet", 
)

Idéer til opgaver:
 - Vis et kort over median indkomsten - hvad kan man se af forskelle mellem fordeling af median indkomst og huspriser?
 - Vis et kort over husenes alder - hvad kan man se af forskelle mellem husenes alder og huspriser?

Del 2: Find sammenhænge i data
---------------------------------------
 - correlation matrix
 - scatter matrix af sammenhænge i data

In [ ]:
housing.plot.scatter(x="median_income", y="median_house_value", alpha=0.1)

In [ ]:
housing.plot.scatter(x="total_rooms", y="median_house_value", alpha=0.1)

In [ ]:
housing.plot.scatter(x="housing_median_age", y="median_house_value", alpha=0.1)

In [ ]:
# Jeg tror det her rer for meget data at overskue for en gymnasieelev på en gang
attributes = ["median_house_value", "median_income", "rooms_per_household",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))

In [ ]:
attributes = ["rooms_per_household","median_income"]
plot.scatter(housing["rooms_per_household"],housing["median_income"])
plot.plot()
scatter_matrix(housing[attributes],figsize=(12, 8))

In [ ]:
ax = housing.plot.scatter(x="rooms_per_household", y="median_house_value", alpha=0.1)
ax.set_xlim(0, 20) # changing axis

In [ ]:
# Dem med lave indkomster lever både færre og flere sammen, mens de rige har meget mindre spredning
ax = housing.plot.scatter(x="population_per_household", y="median_income", alpha=0.1)
ax.set_xlim(0, 10) # changing axis

In [ ]:
# Vi kan bruge correlation matricer til at se sammenhæng mellem dimensionerne
corr_matrix = housing.corr()
corr_matrix

Del 3: Forbered træning og testsæt
------------------------------------------

In [ ]:
test_size = int(len(housing)*0.1)
final_size = int(len(housing)*0.1)
validation_set, rest  = np.split(housing,[test_size])
test_set, training_set = np.split(rest,[test_size])

# burde splittet i labels og data indsættes i biblioteket?
test_data = test_set.drop("median_house_value", axis = 1)
test_labels = test_set["median_house_value"]

validation_data = validation_set.drop("median_house_value", axis = 1)
validation_labels = validation_set["median_house_value"]

training_data = training_set.drop("median_house_value", axis = 1)
training_labels = training_set["median_house_value"]


In [ ]:
scaler = StandardScaler()
scaler.fit(test_data)
scaled_test_data = scaler.transform(test_data)

scaler.fit(validation_data)
scaled_validation_data = scaler.transform(validation_data)

scaler.fit(training_data)
scaled_training_data = scaler.transform(training_data)


Del 4: Træning m. 3 forskellige former for regression
------------------------------------------


In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(scaled_training_data, training_labels)
acc = datanoutlibD1.CalculateAccuracy(scaled_test_data, test_labels,lin_reg) 
acc
# MD: beregningen af RMSE synes jeg ikke skal gemmes væk i biblioteket

In [ ]:
some_data = scaled_training_data[0:5] # MD: hvad er forskellen på det her og housing[0:5] - det sidste måske simplere?
some_labels = training_labels[0:5] # MD: hvad er forskellen på det her og housing_labels[0:5] ?
print("Predictions:", lin_reg.predict(some_data))
print("Labels:", list(some_labels))


In [ ]:
tree_reg = DecisionTreeRegressor()
tree_reg.fit(scaled_training_data, training_labels)
acc = datanoutlibD1.CalculateAccuracy(scaled_validation_data, validation_labels,tree_reg)
print(acc)
# MD: igen, beregningen af RMSE synes jeg ikke skal gemmes væk i biblioteket

In [ ]:
# MD: I stedet for at introducere cross validation, så skal vi måske splitte vores data i 3 sæt fra starten:
# train (70%), test (20%) og validation (10%)? Eller noget i den stil?

tree_cross_val = datanoutlibD1.cross_validate(tree_reg, data_train, labels_train)
datanoutlibD1.display_scores(tree_cross_val)

In [ ]:

lin_scores = datanoutlibD1.cross_validate(lin_reg, data_train, labels_train)
datanoutlibD1.display_scores(lin_scores)

In [ ]:

forest_reg = RandomForestRegressor()
forest_reg.fit(data_train, labels_train)
forest_accuracy = datanoutlibD1.CalculateAccuracy(data_test, labels_test,forest_reg)
print(forest_accuracy)



Del 5: Grid search
------------------

In [ ]:
# ignorer nedad herfra

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor()

# MD: jeg er ikke sikker på jeg synes vi overhovedet skal gå videre med resten her, men måske bare stoppe her
# og så komme mere i dybden med alt det ovenfor. Så S

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
     print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = num_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

In [ ]:

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                          loc=squared_errors.mean(),
                          scale=stats.sem(squared_errors)))